In [1]:
from helpers import chunk_and_contextualize_text 
import chromadb 
import ollama
chroma_client = chromadb.PersistentClient(path="chromadb/")
# chroma_client.delete_collection("my_collection")
collection = chroma_client.create_collection(name="mc_collection", get_or_create=True)

In [2]:
count = 0
def add_to_chroma_db(text: str):
    global count 
    embed = ollama.embeddings(model='nomic-embed-text', prompt=text)
    collection.add(embeddings=[embed["embedding"]], documents=[text], ids=[f"id_{count}"])
    count += 1
def query_chroma_db(text: str, n_results=10):
    return collection.query(ollama.embeddings(model='nomic-embed-text', prompt=text)["embedding"], n_results=n_results)

In [3]:
# %%time

# # find all file in data directory 
# import os
# # iterate over all files in data directory
# # add them into chromadb
# for filename in os.listdir("data"):
#     with open(f"data/{filename}", "r") as f:
#         print(f"Processing {filename}")
#         text = f.read()
#         chunked_text = chunk_and_contextualize_text(text)
#         for chunk in chunked_text:
#             add_to_chroma_db(chunk)

In [4]:
def ask_rag(question, model='24k-llama3.2'):
    responses = query_chroma_db(question, n_results=10)
    # for response in responses['documents'][0]:
    #     print(response)
    context = ""
    for i in range(len(responses['documents'][0])):
        context += f"{i+1}. {responses['documents'][0][i]} "
    
    system = f"""
    You are a Minecraft expert. You are answering a question about the game of Minecraft.
    Use the following pieces of context to answer the question at the end. 
    If you don't know the answer, just say that you don't know, don't try to make up an answer.
    Use five sentences maximum and keep the answer as concise as possible. 
    {context}
    """
            
    response = ollama.chat(model=model, messages=[
    {
        'role': 'system',
        'content': system,
    },
    {
        'role': 'user',
        'content': question,
    },
    ])
    
    return response['message']['content']

In [5]:
print(ask_rag("Where can I find witch?", model='24k-llama3.2'))

I don't know where you can find a witch in Minecraft.


In [6]:
print(ask_rag("How much health points does a Ender Dragon have?", model='24k-llama3.2'))

A Minecraft Ender Dragon has 200 health points.


In [7]:
print(ask_rag("Where can I find Sniffers?", model='24k-llama3.2'))

You can find Sniffers by hatching from sniffer eggs, which can be obtained by breaking suspicious sand blocks found in warm ocean ruins.


In [8]:
print(ask_rag("What does a piglin brute do?", model='24k-llama3.2'))

A piglin brute charges at the player on sight, unlike piglins which barter with players. They wield a golden axe and are considered hostile mob types.
